In [10]:
import pickle
from sklearn import datasets

x =  datasets.make_friedman1(n_samples = 50, n_features = 5, noise = 0)
pickle.dump(x, '~/friedman1')

TypeError: file must have a 'write' attribute

In [16]:
#compare grid and random search. search over all combinations. random forest

'''
note that, for random forest:
n_estimators = num.trees
min_samples_leaf = min.node.size
max_features = mtry
max_samples = sample.fraction
'''

import numpy as np
from sklearn import datasets
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
import time

x =  datasets.make_friedman1(n_samples = 50, n_features = 5, noise = 0)
par_grid = {'n_estimators':[1,2,3,4,5], 
        'min_samples_leaf':np.linspace(0.1,0.5,5), 
        'max_features':np.linspace(0.1,0.5,5), 
        'max_samples':np.linspace(0.1,0.5,5)}

rf = RandomForestRegressor()
gs = model_selection.GridSearchCV(estimator = rf, param_grid = par_grid, cv = 3)
%time gs.fit(x[0], x[1])
print(gs.best_params_)
print(gs.best_score_)

rs = model_selection.RandomizedSearchCV(estimator = rf, 
                                    param_distributions = par_grid,
                                    n_iter = 5**4, 
                                    cv = 3)
                                    
%time rs.fit(x[0], x[1])
print(rs.best_params_)
print(rs.best_score_)

Wall time: 12.2 s
{'max_features': 0.5, 'max_samples': 0.4, 'min_samples_leaf': 0.1, 'n_estimators': 4}
0.19082367577547862
Wall time: 11.5 s
{'n_estimators': 4, 'min_samples_leaf': 0.1, 'max_samples': 0.4, 'max_features': 0.5}
0.2494112665310301


In [ ]:
#compare grid and random search. search along only 1 parameter at a time. random forest.
import numpy as np
from sklearn import datasets
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
import time

x = datasets.make_friedman1(n_samples = 50, n_features = 5, noise = 0)

#I decided to define the default parameters as somewhere in the middle of the entire space
def_pars = {'n_estimators':3, 'min_samples_leaf':0.3, 'max_samples':0.3, 'max_features':0.3}
best_pars = def_pars

param_grid = {'n_estimators':[1,2,3,4,5], 
        'min_samples_leaf':np.linspace(0.1,0.5,5), 
        'max_features':np.linspace(0.1,0.5,5), 
        'max_samples':np.linspace(0.1,0.5,5)}
space_sizes = {'n_estimators':5, 'min_samples_leaf':5, 'max_features':5, 'max_samples':5}

for(par in param_grid):
    rf = RandomForestRegressor(n_estimators = best_pars['n_estimators'],
                                min_samples_leaf = best_pars['min_samples_leaf'],
                                max_samples = best_pars['max_samples'],
                                max_features = best_pars['max_features']
                               )
    gs = model_selection.GridSearchCV(estimator = rf,
                                      param_grid = param_grid,
                                      cv = 3,
                                      scoring = 'neg_mean_squared_error'
                                     )
    gs.fit(x[0], x[1])
    rs = model_selection.RandomizedSearchCV(estimator = rf,
                                           param_distributions = param_grid,
                                           cv = 3,
                                            n_iter = 5
                                           scoring = 'neg_mean_squared_error')
    rs.fit(x[0], x[1])